# Notebook for initial exploration of the auxiliary data

This assumes `initialisation.ipynb` has already been run

In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from pathlib import Path

DATA_PATH = Path("data")
AUXILIARY_DATA_PATH = DATA_PATH / "auxiliary-data"

In [3]:
# Load the cleaned dataset
train_clean = pd.read_csv(DATA_PATH / "train-cleaned.csv")
test_clean = pd.read_csv(DATA_PATH / "test-cleaned.csv")

## Commercial centres

In [4]:
commercial_centres_df = pd.read_csv(AUXILIARY_DATA_PATH / "sg-commerical-centres.csv") # typo in original file name

# Rename lat lng to latitude and longitude
commercial_centres_df.rename(columns={"lat": "latitude", "lng": "longitude"}, inplace=True)
commercial_centres_df.head()

,name,type,latitude,longitude
0,Central Business District,CR,1.286768,103.854529
1,Jurong Lake District,CR,1.334085,103.734513
2,Seletar Regional Centre,CR,1.409435,103.866820
3,Tampines Regional Centre,CR,1.353180,103.945188
4,Woodlands Regional Centre,CR,1.436748,103.786485


There are some issues with the data (e.g. typo in IEBP/IEPB, Tao Payoh etc) but overall it looks fine

We define four types of commercial centres:
- CR: Regional Centres
- IEBP: Industrial Estates and Business Parks
- BN: Business Nodes
- IHL: Institutions of Higher Learning

In [5]:
# Clean the data: replace the IEPB typo with the correct spelling 
commercial_centres_df["type"] = commercial_centres_df["type"].str.replace("IEPB", "IEBP")
commercial_centres_df.type.unique()

array(['CR', 'IEBP', 'BN', 'IHL'], dtype=object)

We can augment the train/test dataframes with the commercial centre information. Since the type of commercial centres are quite different (e.g. IEBP vs IHL), we'll create a new feature for each type of commercial centre. We'll use the Eucledian distance to the nearest commercial centre of each type as the feature value.

Brute forcing takes too long, so we'll use a KDTree to speed things up.

In [6]:
from sklearn.neighbors import BallTree

def distance_series(df, centres):
    """
    Returns a series of distances from each row in df to the nearest centre in centres.
    Assumes that df and centres have columns "latitude" and "longitude".
    """
    # Convert to radians without modifying the original dataframes
    df_rad = np.radians(df.loc[:, ["latitude", "longitude"]])
    centres_rad = np.radians(centres.loc[:, ["latitude", "longitude"]])

    ball = BallTree(centres_rad[["latitude", "longitude"]], metric="haversine")
    distances, _ = ball.query(df_rad[["latitude", "longitude"]], k=1)
    return pd.Series(distances.ravel() * 6371, index=df.index)

In [7]:
from sklearn.neighbors import BallTree

centre_types = set(commercial_centres_df.type)

print("Calculating distances to commercial centres...")
for centre_type in centre_types:
    print(f"Calculating distances to {centre_type}...")
    # Get points of the specific centre type
    points = commercial_centres_df[commercial_centres_df.type == centre_type][["latitude", "longitude"]]
    # Calculate distances
    train_clean["distance_to_" + centre_type] = distance_series(train_clean, points)
    test_clean["distance_to_" + centre_type] = distance_series(test_clean, points)


Calculating distances to commercial centres...
Calculating distances to IHL...
Calculating distances to BN...
Calculating distances to IEBP...
Calculating distances to CR...


In [8]:
train_clean.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,...,elevation,subzone,planning_area,region,resale_price,median_storey,distance_to_IHL,distance_to_BN,distance_to_IEBP,distance_to_CR
0,2001-08-01,pasir ris,4 room,440,pasir ris drive 4,118.0,model a,uncategorized,1989,1.369008,...,18,pasir ris drive,pasir ris,east region,209700.0,2,3.117373,6.552887,2.374153,2.313575
1,2014-10-01,punggol,5 room,196B,punggol field,110.0,improved,uncategorized,2003,1.399007,...,27,punggol field,punggol,north-east region,402300.0,11,8.976871,6.562720,1.462434,4.613532
2,2020-09-01,sengkang,5 room,404A,fernvale lane,112.0,premium apartment,uncategorized,2004,1.388348,...,10,fernvale,sengkang,north-east region,351000.0,2,10.588208,4.128146,1.444920,2.470322
3,2000-10-01,clementi,3 room,375,clementi avenue 4,67.0,new generation,uncategorized,1980,1.318493,...,22,clementi north,clementi,west region,151200.0,8,2.437589,2.921989,2.387090,3.976174
4,2013-01-01,bukit batok,3 room,163,bukit batok street 11,73.0,model a,uncategorized,1985,1.348149,...,23,bukit batok west,bukit batok,west region,318600.0,8,6.671991,7.018465,2.311488,1.807090


## Markets and hawker centers

In [9]:
markets_hawkers_df = pd.read_csv(AUXILIARY_DATA_PATH / "sg-gov-markets-hawker-centres.csv")
markets_hawkers_df.rename(columns={"lat": "latitude", "lng": "longitude"}, inplace=True)
markets_hawkers_df.head()

,name,latitude,longitude
0,East Coast Lagoon Food Village,1.307730,103.934303
1,Jurong West Hawker Centre,1.341223,103.697374
2,Bukit Merah View Blk 115 (Blk 115 Bukit Merah ...,1.285240,103.822372
3,Geylang Bahru Blk 69 (Blk 69 Geylang Bahru Mar...,1.321530,103.870071
4,Ghim Moh Road Blk 20,1.311060,103.788292


In [10]:
# Once again we'll add the distance to the nearest market/hawker centre
train_clean["distance_to_market_hawker"] = distance_series(train_clean, markets_hawkers_df)
test_clean["distance_to_market_hawker"] = distance_series(test_clean, markets_hawkers_df)

In [11]:
train_clean.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,...,subzone,planning_area,region,resale_price,median_storey,distance_to_IHL,distance_to_BN,distance_to_IEBP,distance_to_CR,distance_to_market_hawker
0,2001-08-01,pasir ris,4 room,440,pasir ris drive 4,118.0,model a,uncategorized,1989,1.369008,...,pasir ris drive,pasir ris,east region,209700.0,2,3.117373,6.552887,2.374153,2.313575,0.945560
1,2014-10-01,punggol,5 room,196B,punggol field,110.0,improved,uncategorized,2003,1.399007,...,punggol field,punggol,north-east region,402300.0,11,8.976871,6.562720,1.462434,4.613532,3.782394
2,2020-09-01,sengkang,5 room,404A,fernvale lane,112.0,premium apartment,uncategorized,2004,1.388348,...,fernvale,sengkang,north-east region,351000.0,2,10.588208,4.128146,1.444920,2.470322,1.803941
3,2000-10-01,clementi,3 room,375,clementi avenue 4,67.0,new generation,uncategorized,1980,1.318493,...,clementi north,clementi,west region,151200.0,8,2.437589,2.921989,2.387090,3.976174,0.618292
4,2013-01-01,bukit batok,3 room,163,bukit batok street 11,73.0,model a,uncategorized,1985,1.348149,...,bukit batok west,bukit batok,west region,318600.0,8,6.671991,7.018465,2.311488,1.807090,0.761921


## Population demographics

In [12]:
population_demo_df = pd.read_csv(AUXILIARY_DATA_PATH / "sg-population-demographics.csv")
population_demo_df.head()

,plannin_area,subzone,age_group,sex,count
0,ang mo kio,ang mo kio town centre,0-4,m,130
1,ang mo kio,cheng san,0-4,m,670
2,ang mo kio,chong boon,0-4,m,460
3,ang mo kio,kebun bahru,0-4,m,380
4,ang mo kio,sembawang hills,0-4,m,90


In [13]:
population_demo_df.age_group.unique()

array(['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74',
       '75-79', '80-84', '85+'], dtype=object)

Given that age group is encoded as such: ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+'], lets encode this as the median of the range for easier analysis.

For the final value 85+, we'll assume it's 87.

In [14]:
# Helper functions
def age_bucket(r: str) -> int:
    parts = r.split("-")
    if len(parts) == 1:
        return int(parts[0].rstrip("+")) + 2  # Handle the 85+ case
    else:
        return (int(parts[0]) + int(parts[1])) // 2

# Ensure helper function works as expected
all_ranges = set(population_demo_df["age_group"].unique())
for r in sorted(all_ranges):
    print(f"{r} -> {age_bucket(r)}")



0-4 -> 2
10-14 -> 12
15-19 -> 17
20-24 -> 22
25-29 -> 27
30-34 -> 32
35-39 -> 37
40-44 -> 42
45-49 -> 47
5-9 -> 7
50-54 -> 52
55-59 -> 57
60-64 -> 62
65-69 -> 67
70-74 -> 72
75-79 -> 77
80-84 -> 82
85+ -> 87


In [15]:
population_demo_df["age_bucket"] = population_demo_df["age_group"].apply(age_bucket)
population_demo_df.head()

,plannin_area,subzone,age_group,sex,count,age_bucket
0,ang mo kio,ang mo kio town centre,0-4,m,130,2
1,ang mo kio,cheng san,0-4,m,670,2
2,ang mo kio,chong boon,0-4,m,460,2
3,ang mo kio,kebun bahru,0-4,m,380,2
4,ang mo kio,sembawang hills,0-4,m,90,2


In [16]:
# Can we use this data? Do all subzones in our train/test dataset have corresponding data?
subzones_available_in_demo = set(population_demo_df.subzone)
subzones_present = set(train_clean.subzone) | set(test_clean.subzone)

if subzones_available_in_demo.issuperset(subzones_present):
    print("All subzones have population data")
else:
    print("Not all subzones have population data, missing:")
    print(subzones_present - subzones_available_in_demo)

Not all subzones have population data, missing:
{'city hall'}


In [17]:
# Ah well looks like we cannot group this on a subzone level. What about on a planning area level?
planning_areas_available_in_demo = set(population_demo_df.plannin_area)
planning_areas_present = set(train_clean.planning_area) | set(test_clean.planning_area)

assert planning_areas_available_in_demo.issuperset(planning_areas_present), f"Not all planning areas have population data, missing: {planning_areas_present - planning_areas_available_in_demo}"

Unfortunately we couldn't group on a subzone level because of missing demographic data for city hall. We'll group by the planning area instead.

In [18]:
# Now that we have age buckets, we can group by subzone and calculate the mean age and stddev
# Usually we'd just take the median, but our age data is not very granular
def histogram_mean_std(grouped_data):
    buckets = grouped_data["age_bucket"].values
    counts = grouped_data["count"].values
    total_count = np.sum(counts)

    mean = np.sum(buckets * counts) / total_count
    variance = np.sum(counts * (buckets - mean)**2) / total_count
    std_dev = np.sqrt(variance)

    return pd.Series({"mean_age": mean, "std_age": std_dev})

population_stats_for_area = population_demo_df.groupby(["plannin_area", "sex"]).apply(histogram_mean_std)
population_stats_for_area.head()


mean_age    std_age
plannin_area sex                      
ang mo kio   f    42.592323  21.625228
             m    41.114645  21.358572
bedok        f    41.435397  21.630092
             m    40.054513  21.219524
bishan       f    41.347779  21.244620

Lets merge this data into our train and test datasets. For every row in our train/test dataset, we'll need 4 new columns: `mean_age_m`, `mean_age_f`, `std_age_m`, `std_age_f`. We'll join on the planning area column (note the typo in the original aux dataset)

In [19]:
unstacked = population_stats_for_area.unstack()
unstacked.columns = ["_".join(col).strip() for col in unstacked.columns]
pop_stats_flattened = unstacked.reset_index()
pop_stats_flattened.head()

,plannin_area,mean_age_f,mean_age_m,std_age_f,std_age_m
0,ang mo kio,42.592323,41.114645,21.625228,21.358572
1,bedok,41.435397,40.054513,21.630092,21.219524
2,bishan,41.347779,39.168101,21.244620,21.148499
3,bukit batok,38.980065,38.318241,20.497124,20.287059
4,bukit merah,43.394959,41.766349,22.450599,21.868662


In [20]:
train_clean = train_clean.merge(pop_stats_flattened, how="left", left_on="planning_area", right_on="plannin_area")
train_clean.drop(columns=["plannin_area"], inplace=True)

test_clean = test_clean.merge(pop_stats_flattened, how="left", left_on="planning_area", right_on="plannin_area")
test_clean.drop(columns=["plannin_area"], inplace=True)

## Primary school and Secondary school

We'll merge this in the same way we did the commercial centres data. However, this time we want to merge in the name of the school as well, because the school choice may have a factor in the price.

In [21]:
pri_df = pd.read_csv(AUXILIARY_DATA_PATH / "sg-primary-schools.csv")
pri_df.rename(columns={"lat": "latitude", "lng": "longitude"}, inplace=True)

sec_df = pd.read_csv(AUXILIARY_DATA_PATH / "sg-secondary-schools.csv")
sec_df.rename(columns={"lat": "latitude", "lng": "longitude"}, inplace=True)

In [22]:
# List duplicates in the name column
sec_df.duplicated(subset=["name"]).sum()

0

In [23]:
# We'll need a modification of our distance_series function to also return the name
def distance_series_with_name(df, pois):
    """
    Returns a series of distances from each row in df to the nearest centre in centres.
    Assumes that df and centres have columns "latitude" and "longitude".
    """
    # Convert to radians without modifying the original dataframes
    df_rad = np.radians(df.loc[:, ["latitude", "longitude"]])
    centres_rad = np.radians(pois.loc[:, ["latitude", "longitude"]])

    ball = BallTree(centres_rad[["latitude", "longitude"]], metric="haversine")
    distances, indices = ball.query(df_rad[["latitude", "longitude"]], k=1)
    
    # Get the corresponding "name" values using numpy
    names = pois['name'].to_numpy()[indices.ravel()]

    # Create a new dataframe with distance and name columns
    result = pd.DataFrame({'distance': distances.ravel() * 6371, 'name': names}, index=df.index)
    return result

# Calculate distances to primary schools
train_clean[["pri_sch_dist", "pri_sch"]] = distance_series_with_name(train_clean, pri_df)
test_clean[["pri_sch_dist", "pri_sch"]] = distance_series_with_name(test_clean, pri_df)

# Calculate distances to secondary schools
train_clean[["sec_sch_dist", "sec_sch"]] = distance_series_with_name(train_clean, sec_df)
test_clean[["sec_sch_dist", "sec_sch"]] = distance_series_with_name(test_clean, sec_df)

In [24]:
train_clean.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,...,distance_to_CR,distance_to_market_hawker,mean_age_f,mean_age_m,std_age_f,std_age_m,pri_sch_dist,pri_sch,sec_sch_dist,sec_sch
0,2001-08-01,pasir ris,4 room,440,pasir ris drive 4,118.0,model a,uncategorized,1989,1.369008,...,2.313575,0.945560,37.379398,36.167630,20.331631,19.999478,0.344087,Loyang Primary School,0.428301,Pasir Ris Crest Secondary School
1,2014-10-01,punggol,5 room,196B,punggol field,110.0,improved,uncategorized,2003,1.399007,...,4.613532,3.782394,32.775862,31.967676,20.103889,19.793305,0.160852,Edgefield Primary School,0.312383,Meridian Secondary School
2,2020-09-01,sengkang,5 room,404A,fernvale lane,112.0,premium apartment,uncategorized,2004,1.388348,...,2.470322,1.803941,35.050443,34.164736,20.311337,19.947820,0.184906,Fernvale Primary School,0.558380,Pei Hwa Secondary School
3,2000-10-01,clementi,3 room,375,clementi avenue 4,67.0,new generation,uncategorized,1980,1.318493,...,3.976174,0.618292,42.178870,40.577282,21.625967,21.440329,0.304561,Pei Tong Primary School,0.619132,Clementi Town Secondary School
4,2013-01-01,bukit batok,3 room,163,bukit batok street 11,73.0,model a,uncategorized,1985,1.348149,...,1.807090,0.761921,38.980065,38.318241,20.497124,20.287059,0.233809,Princess Elizabeth Primary School,0.217911,Bukit Batok Secondary School


## Shopping malls

They're basically all the same, we'll just add a single column to the train/test dataframes for distance to the nearest shopping mall.

In [25]:
shopping_malls = pd.read_csv(AUXILIARY_DATA_PATH / "sg-shopping-malls.csv")
shopping_malls.rename(columns={"lat": "latitude", "lng": "longitude"}, inplace=True)
shopping_malls.drop(columns=["wikipedia_link"], inplace=True)

# We can re-use our distance_series function to calculate distances to shopping malls
train_clean["mall_dist"] = distance_series(train_clean, shopping_malls)
test_clean["mall_dist"] = distance_series(test_clean, shopping_malls)

train_clean.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,...,distance_to_market_hawker,mean_age_f,mean_age_m,std_age_f,std_age_m,pri_sch_dist,pri_sch,sec_sch_dist,sec_sch,mall_dist
0,2001-08-01,pasir ris,4 room,440,pasir ris drive 4,118.0,model a,uncategorized,1989,1.369008,...,0.945560,37.379398,36.167630,20.331631,19.999478,0.344087,Loyang Primary School,0.428301,Pasir Ris Crest Secondary School,1.033216
1,2014-10-01,punggol,5 room,196B,punggol field,110.0,improved,uncategorized,2003,1.399007,...,3.782394,32.775862,31.967676,20.103889,19.793305,0.160852,Edgefield Primary School,0.312383,Meridian Secondary School,0.806040
2,2020-09-01,sengkang,5 room,404A,fernvale lane,112.0,premium apartment,uncategorized,2004,1.388348,...,1.803941,35.050443,34.164736,20.311337,19.947820,0.184906,Fernvale Primary School,0.558380,Pei Hwa Secondary School,0.452556
3,2000-10-01,clementi,3 room,375,clementi avenue 4,67.0,new generation,uncategorized,1980,1.318493,...,0.618292,42.178870,40.577282,21.625967,21.440329,0.304561,Pei Tong Primary School,0.619132,Clementi Town Secondary School,0.456499
4,2013-01-01,bukit batok,3 room,163,bukit batok street 11,73.0,model a,uncategorized,1985,1.348149,...,0.761921,38.980065,38.318241,20.497124,20.287059,0.233809,Princess Elizabeth Primary School,0.217911,Bukit Batok Secondary School,0.764172


## MRT stations

This is a lot trickier, as when a sale happened, a given MRT in the vicinity may not have been completed yet. We'll need to carefully query by buckets to ensure only valid MRT stations for that given dates are included in the nearest MRT station calculation.

In [26]:
mrt_stations = pd.read_csv(AUXILIARY_DATA_PATH / "sg-train-stations.csv")
mrt_stations.rename(columns={"lat": "latitude", "lng": "longitude"}, inplace=True)

# For sanity sake, we'll drop the lrt/mrt type columns, as well as the codes columns
mrt_stations.drop(columns=["type", "codes"], inplace=True)

# There seem to be NaNs in the data for opening_year
print("Number of stations with no year:", mrt_stations.opening_year.isna().sum())
print("List of stations with no year:")
mrt_stations[mrt_stations.opening_year.isna()]

Number of stations with no year: 40
List of stations with no year:


,name,latitude,longitude,opening_year
40,Oasis,1.402286,103.912727,NaN
41,Sam Kee,1.409612,103.904831,NaN
42,Farmway,1.397170,103.889304,NaN
43,Kupang,1.398212,103.881256,NaN
44,Renjong,1.386723,103.890539,NaN
46,Fajar,1.384520,103.770808,NaN
48,Samudera,1.415901,103.902156,NaN
49,Nibong,1.411870,103.900313,NaN
50,Soo Teck,1.405088,103.897209,NaN
51,Kadaloor,1.399584,103.916486,NaN


In [27]:
# Ah that's annoying... I found a separate dataset, lets merge it in
supplementary_mrts = pd.read_csv(Path("supplementary") / "mrt_lrt_data.csv")

# For every NaN in opening_year, we'll try to find a value in the supplementary dataset
# Easier to do this inefficiently in a for loop for small datasets
for index, row in mrt_stations[mrt_stations.opening_year.isna()].iterrows():
    name = row["name"]
    if name in supplementary_mrts.station_name.values:
        mrt_stations.at[index, "opening_year"] = supplementary_mrts[supplementary_mrts.station_name == name].operational_date.values[0].split('-')[0]

print("Number of stations with no year:", mrt_stations.opening_year.isna().sum())
print("List of stations with no year:")
mrt_stations[mrt_stations.opening_year.isna()]

Number of stations with no year: 2
List of stations with no year:


,name,latitude,longitude,opening_year
78,Ten Mile Junction,1.380320,103.760139,NaN
105,Bukit Brown,1.333728,103.830689,NaN


Only 2 MRT stations are missing, so we'll just drop them. Ten Mile has a complicated opening/closing/reopening history, and Bukit Brown hasn't been opened yet.

In [28]:
mrt_stations.dropna(inplace=True)
print("Number of stations with no year:", mrt_stations.opening_year.isna().sum())

Number of stations with no year: 0


In [29]:
# Lastly, we'll convert the year to a datetime object
# For some reason, a decimal point got added to the end of the year, so we'll remove it
mrt_stations["opening_year"] = mrt_stations["opening_year"].astype(str).str.replace(r"\.0$", "", regex=True)
mrt_stations["opening_year"] = pd.to_datetime(mrt_stations["opening_year"], format="%Y")
mrt_stations.head()

,name,latitude,longitude,opening_year
0,Eunos,1.319778,103.903252,1989-01-01
1,Chinese Garden,1.342352,103.732596,1988-01-01
2,Khatib,1.417383,103.832980,1988-01-01
3,Kranji,1.425177,103.762165,1996-01-01
4,Redhill,1.289562,103.816816,1988-01-01


To approach this problem of finding the nearest station given the opening_year and the date of sale, we first bucket the MRTs into date_buckets, then iterate over them in reverse order from the most recently opened station to the oldest.

In other words:
- Find all unique `opening_year` and sort them
- Iterate in reverse order
  - For a newer date, all MRTs that opened before that date are valid. For this set of valid MRTs, sales that happened after that date but before the previous bucket (since those will have already been filled with a larger set of valid MRTs) are valid.
  - As we iterate on older and older dates, less and less MRTs become valid.

In [30]:
# Convert the train/test month field to a datetime object. 
# We'll need to remember to delete this column later when saving.
train_clean["month_dt"] = pd.to_datetime(train_clean["month"], format="%Y-%m-%d")
test_clean["month_dt"] = pd.to_datetime(test_clean["month"], format="%Y-%m-%d")

In [31]:
def fill_nearest_mrt(df, df_mask, mrts, mrts_mask):
    # Convert to radians without modifying the original dataframes
    df_rad = np.radians(df.loc[df_mask, ["latitude", "longitude"]])
    mrts_rad = np.radians(mrts.loc[mrts_mask, ["latitude", "longitude"]])

    ball = BallTree(mrts_rad[["latitude", "longitude"]], metric="haversine")
    distances, indices = ball.query(df_rad[["latitude", "longitude"]], k=1)
    df.loc[df_mask, "mrt_name"] = mrts.loc[mrts_mask].iloc[indices.flatten()]["name"].values
    df.loc[df_mask, "mrt_dist"] = distances.ravel() * 6371

def fill_by_bucket(df, mrts):
    """
    Note that unlike the previous functions, this WILL modify the input df for sanity sake.
    """
    date_buckets = np.sort(mrts["opening_year"].unique())
    to_bucket = pd.Timestamp.max

    for from_bucket in date_buckets[::-1]:
        candidate_mrts_mask = (mrts["opening_year"] <= from_bucket)
        candidate_sales_mask = (df["month_dt"] >= from_bucket) & (df["month_dt"] < to_bucket)

        if not df.loc[candidate_sales_mask].empty:
            fill_nearest_mrt(df, candidate_sales_mask, mrts, candidate_mrts_mask)
        
        to_bucket = from_bucket


In [32]:
fill_by_bucket(train_clean, mrt_stations)
fill_by_bucket(test_clean, mrt_stations)

train_clean[["latitude", "longitude", "town", "mrt_name", "mrt_dist"]].head()

,latitude,longitude,town,mrt_name,mrt_dist
0,1.369008,103.958697,pasir ris,Pasir Ris,1.137522
1,1.399007,103.906991,punggol,Cove,0.118373
2,1.388348,103.873815,sengkang,Fernvale,0.481153
3,1.318493,103.766702,clementi,Clementi,0.423320
4,1.348149,103.742658,bukit batok,Bukit Batok,0.774220


In [33]:
test_clean.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,...,std_age_f,std_age_m,pri_sch_dist,pri_sch,sec_sch_dist,sec_sch,mall_dist,month_dt,mrt_name,mrt_dist
0,2004-01-01,bukit batok,4 room,186,bukit batok west avenue 6,94.0,new generation,uncategorized,1989,1.346581,...,20.497124,20.287059,0.467109,Princess Elizabeth Primary School,0.426327,Bukit Batok Secondary School,0.693942,2004-01-01,Bukit Batok,0.667549
1,2001-11-01,tampines,5 room,366,tampines street 34,122.0,improved,uncategorized,1997,1.357618,...,20.670669,20.366271,0.530017,East Spring Primary School,0.078892,East Spring Secondary School,1.933979,2001-11-01,Simei,1.833500
2,2002-07-01,jurong east,3 room,206,jurong east street 21,67.0,new generation,uncategorized,1982,1.337804,...,20.942785,20.636231,0.577345,Yuhua Primary School,0.401047,Shuqun Secondary School,0.524612,2002-07-01,Jurong East,0.518257
3,2015-04-01,ang mo kio,3 room,180,ang mo kio avenue 5,82.0,new generation,uncategorized,1981,1.380084,...,21.625228,21.358572,0.800015,Mayflower Primary School,0.571233,Anderson Secondary School,0.829128,2015-04-01,Yio Chu Kang,0.546923
4,2004-04-01,clementi,5 room,356,clementi avenue 2,117.0,standard,uncategorized,1978,1.313960,...,21.625967,21.440329,0.316899,Pei Tong Primary School,0.443189,"School of Science and Technology, Singapore",0.604664,2004-04-01,Clementi,0.515878


In [34]:
# remember to drop the month_dt column
train_clean.drop(columns=["month_dt"], inplace=True)
test_clean.drop(columns=["month_dt"], inplace=True)

# Save our augmented data to disk under DATA_PATH / "train-augmented.csv" and DATA_PATH / "test-augmented.csv"
train_clean.to_csv(DATA_PATH / "train-augmented.csv", index=False)
test_clean.to_csv(DATA_PATH / "test-augmented.csv", index=False)